In [1]:
# Good Sourde https://forest-benchmarking.readthedocs.io/en/latest/superoperator_representations.html
import numpy as np
import scipy as sp
from qutip import *
from qutip.qip.device import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit
from cvqaoa import *
from cvqaoa.gates import carb
from forest.benchmarking.operator_tools import *
from forest.benchmarking.operator_tools import *
from forest.benchmarking.operator_tools.project_superoperators import *

In [22]:
def product(*args, repeat=1):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x + [y] for x in result for y in pool]
    for prod in result:
        yield list(prod)


N = 2  # qubits
d = 4  # dimension
tau = 2  # gate time
# Load the average gate fidelity
file = np.load('data/average_gate_fidelity/cv_avg_fid_zz.npz')
f_bar = np.mean(file['avg'])
# Find the corresponding T1
gamma = (d + 1) / (d * tau) * (1 - f_bar)
T1 = 1 / (gamma)
# Qubit Processor
qp = QubitProcessor(N=N, T1=T1)
qp_inv = QubitProcessor(N=N, T1=None)

# Pauli matrices
P = [qeye(2), sigmax(), sigmay(), sigmaz()]

# Create all tensor products
sigma = list(map(tensor, product(P, repeat=2)))

# List with angles of rotation
arg_list = np.linspace(0, np.pi, num=181, endpoint=False)

# Initialize kraus list
kraus_list = []

arg = 0.2429961168522492
# Pauli transfer matrix
R = np.zeros((d**2, d**2))

# Quantum circuit
qc = QubitCircuit(N)
qc.user_gates = {"CARB": carb}
qc.add_gate("CARB", targets=[0, 1], arg_value=arg)

# Get propagator
U = (-1j*tensor(sigmaz(),sigmaz())*arg/2).expm()

# Create PTM
opt = Options(nsteps=1e8)
for j in range(d**2):
    eVals, eVecs = sp.linalg.eigh(sigma[j])
    Lambda_tot = 0
    for idx, eVal in enumerate(eVals):
        init_state = ket2dm(Qobj(eVecs[:,idx],dims=[[2,2],[1,1]]))
        result = qp.run_state(init_state=init_state, qc=qc, options=opt)
        final_dm = result.states[-1]
        if abs(final_dm.tr()-1) > 0.00001 or final_dm.isherm == False:
            print(1/0)
        Lambda_tot += eVal * final_dm 
    for i in range(d**2):
        R[i, j] = 1 / d * ((sigma[i] * Lambda_tot).tr()).real

# Convert PTM to choi
choi = 1 / d**2 * sum((R[i, j] * tensor(sigma[j].trans(), sigma[i]))
                      for i in range(d**2) for j in range(d**2))
choi.dims = [[[2, 2], [2, 2]], [[2, 2], [2, 2]]]
choi.superrep = 'choi'
choi = choi.full()

# Convert Choi to Kraus
kraus = choi2kraus(choi)

# Get the kraus errorrs
kraus_err = [np.sqrt(d)*k@(U.dag()).full() for k in kraus]

# Check that the kraus sum to identity
iden = sum(np.conj(k.T)@k for k in kraus_err)
if np.isclose(iden, np.eye(4), rtol=1e-10, atol=1e-4).all() != True:
    print(iden)
    print('theta', arg)
print('Finished')

[[ 1.00011981e+00+0.00000000e+00j -1.44812818e-17+1.06144740e-16j
   3.69314315e-21-5.12308583e-21j -2.96008665e-20+4.70783819e-19j]
 [-1.44812818e-17-1.06144740e-16j  1.00004035e+00+0.00000000e+00j
  -1.72402280e-23+1.40539088e-23j -2.30979491e-18-4.88146020e-19j]
 [ 3.69314315e-21+5.12308583e-21j -1.72402280e-23-1.40539088e-23j
   1.00004035e+00+0.00000000e+00j -1.58076847e-18+2.24999727e-18j]
 [-2.96008665e-20-4.70783819e-19j -2.30979491e-18+4.88146020e-19j
  -1.58076847e-18-2.24999727e-18j  1.00012207e+00+0.00000000e+00j]]
theta 0.2429961168522492
Finished


In [3]:
chi = choi2chi(choi)

In [14]:
# Get chi error
V = np.zeros((d**2, d**2),dtype="complex128")
for m in range(d**2):
    for n in range(d**2):
        V[m,n] = (sigma[m].dag()*sigma[n]*U.dag()).tr() / d
chi_err = V@chi@np.conj(V.T)

In [21]:
kraus_err = chi2kraus(chi_err)
iden = sum(np.conj(k.T)@k for k in kraus_err)
4*iden

array([[ 1.00011981e+00+0.00000000e+00j, -1.87073647e-16+1.06158556e-16j,
         3.70936042e-21-1.54178977e-20j,  2.10120608e-19+3.84272826e-20j],
       [-1.87073647e-16-1.06158556e-16j,  1.00004035e+00+0.00000000e+00j,
         1.42115920e-35-2.60208521e-18j, -2.55133831e-18+1.16729323e-19j],
       [ 3.70936042e-21+1.54178977e-20j,  1.42115920e-35+2.60208521e-18j,
         1.00004035e+00+0.00000000e+00j, -1.33193556e-18+2.16471607e-18j],
       [ 2.10120608e-19-3.84272826e-20j, -2.55133831e-18-1.16729323e-19j,
        -1.33193556e-18-2.16471607e-18j,  1.00012207e+00+0.00000000e+00j]])

In [ ]:
eValues, eVectors = sp.linalg.eigh(choi)
print('CP',eValues>0)
#eValues = np.where(eValues < 1e-8, 0, eValues)
rtol = 1e-8
idx, = np.where(eValues < rtol)
eVectors = np.delete(eVectors, idx, axis=1)  # drop columns
eValues = np.delete(eValues, idx)
num = len(eValues)

# Get the Kraus operators
kraus = [np.sqrt(d * eValues[i]) * eVectors[:, i].reshape((d, d))
         for i in range(num)